In [1]:
import h5py
import pandas as pd
import numpy as np
import time

from sklearn.model_selection import train_test_split
import tensorflow as tf

/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anrunw/anaconda3/envs/tfgpu/lib/python3.6/s

### Go Through File and Create Dataframes

In [68]:
f = h5py.File('processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_withPars_truth_0.z','r')
treeArray = f['t_allpar_new'][()]

features = ['j1_etarel','j1_phirel','j1_pt','j1_e','j1_ptrel','j1_erel','j1_deltaR','j_index']
featuresUsed = ['j1_ptrel', 'j1_etarot', 'j1_phirot', 'j1_erel', 'j1_deltaR', 'j1_pdgid','j_index']
pdgid = ['j_index', 'j1_pdgid']
glob = ['j_mass_mmdt', 'j_mass' ,'j_index']
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_index']

features_df = pd.DataFrame(treeArray,columns=list(set(features)))


In [69]:
features_df

,j1_phirel,j1_erel,j1_pt,j1_deltaR,j1_etarel,j1_e,j1_ptrel,j_index
0,0.005412,0.382772,324.363037,0.011056,-0.009641,670.573242,0.386118,101333
1,0.008571,0.105703,90.503120,0.023104,-0.021455,185.180466,0.107734,101333
2,0.004479,0.094339,79.580070,0.006306,-0.004439,165.272293,0.094731,101333
3,0.001903,0.042913,35.354923,0.022473,0.022392,75.178345,0.042086,101333
4,-0.004625,0.038547,33.108334,0.025492,-0.025069,67.530716,0.039412,101333
...,...,...,...,...,...,...,...,...
5131608,-0.368169,0.000344,0.419624,0.380018,-0.094157,0.419657,0.000350,300123134
5131609,0.202120,0.000256,0.309434,0.294476,-0.214157,0.312155,0.000258,300123134
5131610,0.217128,0.000215,0.262346,0.255231,-0.134157,0.262708,0.000219,300123134
5131611,-0.288128,0.000115,0.139532,0.319973,-0.139157,0.139763,0.000116,300123134


In [70]:
features_df['j_index'].drop_duplicates()

0             101333
51         100073064
101        300177633
142        100188310
161        200044159
             ...    
5131388    300173707
5131417    200033311
5131447    400153985
5131486    300037743
5131530    300123134
Name: j_index, Length: 98769, dtype: int32

### Group Features in Dataframe by Jet Index

In [71]:
index_df = features_df.groupby(['j_index'])

In [72]:
index_df.first()

,j1_phirel,j1_erel,j1_pt,j1_deltaR,j1_etarel,j1_e,j1_ptrel
j_index,,,,,,,
1,-0.024659,0.525633,573.877441,0.024813,-0.002755,621.589478,0.526719
3,-0.033144,0.699932,912.233521,0.057165,-0.046575,1025.121826,0.689462
7,0.060146,0.388123,634.409241,0.079966,-0.052698,637.664490,0.390835
21,-0.019943,0.130639,177.829224,0.044376,-0.039642,210.623428,0.133965
23,0.061119,0.129065,151.889832,0.061479,0.006637,152.321976,0.130030
...,...,...,...,...,...,...,...
400199939,-0.075494,0.068866,94.362823,0.226768,-0.213833,116.092384,0.079498
400199945,0.059135,0.158689,169.539261,0.061562,0.017117,172.701096,0.162039
400199962,-0.250765,0.063112,66.630859,0.262295,-0.076911,99.012939,0.067352


In [36]:
index_df.last()

,j1_phirel,j1_erel,j1_pt,j1_deltaR,j1_etarel,j1_e,j1_ptrel
j_index,,,,,,,
1,0.167938,4.914950e-15,5.371203e-12,0.167938,-0.000255,5.812193e-12,4.929822e-15
3,-0.215369,1.716514e-04,2.306876e-01,0.216846,0.025269,2.514011e-01,1.743528e-04
7,-0.294880,2.267890e-15,3.668080e-12,0.371562,0.226063,3.726016e-12,2.259763e-15
21,-0.047145,1.758091e-14,2.550655e-11,0.176887,-0.170488,2.834499e-11,1.921491e-14
23,0.072032,1.724079e-04,2.017970e-01,0.210257,-0.197533,2.034744e-01,1.727544e-04
...,...,...,...,...,...,...,...
400199939,0.237563,1.702099e-04,2.254791e-01,0.285076,-0.157582,2.869349e-01,1.899589e-04
400199945,0.183578,1.999639e-13,2.072527e-10,0.211512,-0.105054,2.176203e-10,1.980837e-13
400199962,0.303338,1.746291e-04,1.612507e-01,0.318107,0.095802,2.739642e-01,1.629955e-04


In [37]:
max_particles = 30

In [60]:
num_features = 7

### Iterate through dataframe to create a numpy array with desired dimensions


In [74]:
data = np.zeros([98769,max_particles,num_features])
p = 0
for name, group in index_df:
    x = group.to_numpy()
    x = np.delete(x,num_features,1)
    g = 0
    for l in x:
        if g == max_particles or g == len(x): 
            break
        for n in range(num_features):
            if n == 1 or n ==2 or n == 5 or n == 6:
                l[n] = np.log(l[n])
            data[p,g,n] = l[n]
        g = g+1

    p = p+1
    

In [76]:
data[0]

array([[-2.46594101e-02, -6.43151844e-01,  6.35241586e+00,
         2.48127822e-02, -2.75455974e-03,  6.43227987e+00,
        -6.41088355e-01],
       [-2.93168835e-02, -2.50687327e+00,  4.48875989e+00,
         2.94277072e-02, -2.55153188e-03,  4.56855837e+00,
        -2.50474436e+00],
       [-2.02614833e-02, -2.62737331e+00,  4.36823665e+00,
         2.04328783e-02, -2.64100265e-03,  4.44805833e+00,
        -2.62526762e+00],
       [-3.46643962e-02, -2.75070819e+00,  4.24097486e+00,
         3.69364135e-02, -1.27545502e-02,  4.32472353e+00,
        -2.75252938e+00],
       [-2.46068090e-02, -3.22349637e+00,  3.76440712e+00,
         3.28626633e-02, -2.17821002e-02,  3.85193534e+00,
        -3.22909721e+00],
       [-2.50142440e-02, -3.28693409e+00,  3.71071245e+00,
         2.51597520e-02,  2.70197680e-03,  3.78849762e+00,
        -3.28279183e+00],
       [-3.17640938e-02, -3.93783934e+00,  3.05365568e+00,
         3.43921594e-02, -1.31857125e-02,  3.13759234e+00,
        -3.9398486

### Create and group labels dataframe by index

In [32]:
labels_df = pd.DataFrame(treeArray,columns=list(set(labels)))


In [33]:
labels_df

,j_g,j_q,j_w,j_index,j_t,j_z
0,1,0,0,101333,0,0
1,1,0,0,101333,0,0
2,1,0,0,101333,0,0
3,1,0,0,101333,0,0
4,1,0,0,101333,0,0
...,...,...,...,...,...,...
5131608,0,0,0,300123134,0,1
5131609,0,0,0,300123134,0,1
5131610,0,0,0,300123134,0,1
5131611,0,0,0,300123134,0,1


In [34]:
labels_df = labels_df.groupby('j_index')

### Iterate through Labels Dataframe to return Numpy Array of labels

In [35]:
labels_df.first()

,j_g,j_q,j_w,j_t,j_z
j_index,,,,,
1,1,0,0,0,0
3,1,0,0,0,0
7,1,0,0,0,0
21,1,0,0,0,0
23,1,0,0,0,0
...,...,...,...,...,...
400199939,0,0,0,1,0
400199945,0,0,0,1,0
400199962,0,0,0,1,0


In [41]:
labels = np.zeros([98769,5])
o = 0
for name,  group in labels_df:
    x = group.to_numpy()
    x = np.delete(x,3,1)
    for z in range(5):
        labels[o,z] = x[0][z]

    o = o+1

In [50]:
np.amax(x)

1

### Group global features by index and iterate to return a numpy array in desired shape

In [51]:
globdf = pd.DataFrame(treeArray,columns = glob)

In [52]:
globdf = globdf.groupby('j_index')

In [53]:
glob = np.zeros([98769, 2])
o = 0
for name, group in globdf:
    x = group.to_numpy()
    x = np.delete(x,2,1)
    for m in range(2):
        glob[o,m] = x[0,m]
    o = o+1

In [54]:
glob


array([[ 90.77446747,  93.24835968],
       [148.98461914, 150.47558594],
       [241.29042053, 241.29042053],
       ...,
       [182.7910614 , 182.7910614 ],
       [145.66259766, 145.66259766],
       [166.74099731, 166.74099731]])

In [55]:
labels.shape

(98769, 5)

### Split into train and validation/test sets

In [56]:
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.50, random_state=42)


In [57]:
trash_train, trash_test, glob_train, glob_test = train_test_split(data, glob, test_size=0.50, random_state=42)

In [58]:
labels_train.shape

(49384, 5)

In [59]:
data_train.shape

(49384, 20, 6)

In [60]:
glob_train.shape

(49384, 2)

In [61]:
data_test.shape

(49385, 20, 6)

### Create File

In [62]:
f = h5py.File('trainABC6feat5tag.h5','w')

In [63]:
f.create_dataset('data', data = data_train)

<HDF5 dataset "data": shape (49384, 20, 6), type "<f8">

In [64]:
f.create_dataset('pid', data = labels_train)

<HDF5 dataset "pid": shape (49384, 5), type "<f8">

In [65]:
f.create_dataset('global', data = glob_train)

<HDF5 dataset "global": shape (49384, 2), type "<f8">

In [66]:
f.keys()


<KeysViewHDF5 ['data', 'global', 'pid']>

In [67]:
f.close()

### Split rest of data into the validation and test sets

In [68]:
data_val, datatest, labels_val, labels_test = train_test_split(data_test, labels_test, test_size=0.50, random_state=42)


In [69]:
trash_val, trash_test, glob_val, glob_test = train_test_split(data_test, glob_test, test_size = 0.50, random_state=42)

### Create Files

In [70]:
f = h5py.File('testABC6feat5tag.h5', 'w')

In [71]:
f.create_dataset('data', data = datatest)


<HDF5 dataset "data": shape (24693, 20, 6), type "<f8">

In [72]:
f.create_dataset('pid', data = labels_test)


<HDF5 dataset "pid": shape (24693, 5), type "<f8">

In [73]:
f.create_dataset('global', data = glob_test)

<HDF5 dataset "global": shape (24693, 2), type "<f8">

In [74]:
f.close()

In [75]:
f = h5py.File('valABC6feat5tag.h5', 'w')

In [76]:
f.create_dataset('data', data = data_val)


<HDF5 dataset "data": shape (24692, 20, 6), type "<f8">

In [77]:
f.create_dataset('pid', data = labels_val)


<HDF5 dataset "pid": shape (24692, 5), type "<f8">

In [78]:
f.create_dataset('global', data = glob_val)

<HDF5 dataset "global": shape (24692, 2), type "<f8">

In [79]:
f.close()

In [80]:
labels

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [111]:
f = h5py.File('trainABC6feat5tag.h5','r')

In [113]:
f['pid'][:][:,0].shape

(49384,)